# Coursera Capstone Project Notebook

### This is a Jupyter Notebook for the Coursera Capstone Project.

In [5]:
import pandas as pd
import numpy as np

In [79]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Import requests, BeautifulSoup and geocoder.

In [1]:
import requests
from bs4 import BeautifulSoup

### Set up URL, loop counter and column array variables.

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

i = 0
postcode = []
borough = []
neighbourhood = []

### Loop through wikipedia table and scrape values. Replace 'Not assigned' neighbourhoods with Borough and merge neighbourhoods under the same postal code.

In [3]:
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    lat_lng_coords = None
    data = items.find_all(['th','td'])
    try:
        if i > 0:
            if data[0].text == postcode[i-1]:
                neighbourhood[i-1] += ','
                neighbourhood[i-1] += data[2].text.strip()
            elif data[2].text.strip() == 'Not assigned':
                postcode += [data[0].text]
                borough += [data[1].text]
                neighbourhood += [data[1].text]
                i+=1
            else:
                postcode += [data[0].text]
                borough += [data[1].text]
                neighbourhood += [data[2].text.strip()]
                i+=1
        else:
            postcode += [data[0].text]
            borough += [data[1].text]
            neighbourhood += [data[2].text.strip()]
            i+=1
    except IndexError:pass

### Load GPS coordinates from CSV file

In [12]:
geo = pd.read_csv('https://cocl.us/Geospatial_data')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Drop rows with 'Not assigned' borough and reset index. Merge with GPS coordinates.

In [16]:
d = {'Postal Code':postcode,'Borough':borough, 'Neighbourhood':neighbourhood}
df = pd.DataFrame(d)
df = df[df.Borough != 'Not assigned']
df = df.reset_index()
df = df.drop(['index'], axis=1)
df = df.sort_values(by=['Postal Code'])
df = pd.merge(df, geo[['Postal Code', 'Latitude', 'Longitude']], on='Postal Code')
df.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [7]:
df.shape

(103, 3)